# 找到内容最相关的三个文档

In [1]:
question = ["福昕软件在全球PDF行业中的竞争地位如何？"]

In [2]:
import pandas as pd

df = pd.read_excel('comment.xlsx')
filename_list = df.iloc[:,0].tolist()
comment_list = df.iloc[:,1].tolist()

# print(comment_list)
# print(filename_list)

comment_list = [str(comment) for comment in comment_list]
filename_list = [str(filename) for filename in filename_list]

# print(comment_list)
# print(filename_list)



In [3]:
import torch

from FlagEmbedding import FlagModel
model = FlagModel('BAAI/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True)
print("模型加载")
file_embeddings = model.encode(comment_list)

d:\anaconda\envs\AI_agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


模型加载


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
d:\anaconda\envs\AI_agent\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
# -*- coding: utf-8 -*-
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import numpy as np
import faiss


nlist = 1  # 聚类中心的数量
quantizer = faiss.IndexFlatL2(1024)  # 基础索引
faiss_index = faiss.IndexIVFFlat(quantizer, 1024, nlist, faiss.METRIC_L2)


faiss_index.train(file_embeddings)
faiss_index.add(file_embeddings)


embeddings_question = model.encode(question)
print(embeddings_question.shape)

distances, doc_indices = faiss_index.search(embeddings_question, 5)

data = []
print(f"query: {question}")

for i in range(doc_indices.shape[0]):
    print('==== query:',question[i])
    for j, sent_index in enumerate(doc_indices[i]):
        print(f"rank: {j}, distance: {distances[i][j]}, text: {comment_list[sent_index]}")
        data.append(filename_list[sent_index])

data = list(set(data))

print(data)

(1, 1024)
query: ['福昕软件在全球PDF行业中的竞争地位如何？']
==== query: 福昕软件在全球PDF行业中的竞争地位如何？
rank: 0, distance: 0.6025930643081665, text: PDF作为一种电子文档格式，在不同设备和操作系统间保持一致性，广泛应用于电子文档、工程制图、出版印刷等领域。2024年行业的潜在市场规模约320亿美元，全球仅少数软件企业掌握完整的PDF格式生成、渲染、转换与版面识别等关键技术。Adobe为绝对龙头，福昕在二线厂商中处于领先位置。
rank: 1, distance: 0.6433514356613159, text: 福昕软件专注于PDF领域，产品矩阵完备，具有自主知识产权体系，被评选为“2024全球最佳软件公司”。公司海外收入占比近九成，客户以机构为主，得到众多国际知名企业认可。公司积极推进订阅制转型，订阅业务ARR达3.25亿元，续费率达94%。
rank: 2, distance: 0.8095030784606934, text: 福昕软件的核心成长逻辑在于对龙头Adobe份额的替代，通过产品矩阵及产品功能的替代、渠道改革和性价比优势实现客户转化。公司依靠云计算技术实现订阅制转型，依靠人工智能技术重塑产品，打开增量空间，增强客户粘性。预计公司2024-2026年归母净利润分别为-0.52/-0.26/0.54亿元，首次覆盖给予“强烈推荐”评级。
rank: 3, distance: 1.1847501993179321, text: 南威软件在2023年面临宏观经济放缓和政府财政压力加大的背景下，仍然实现了新签合同总额25.37亿元，同比增长4.92%。尽管营业收入和净利润有所下降，但公司通过优化销售结构和聚焦核心产品，保持了经营基本面。销售毛利润同比上涨，显示出公司在市场中的竞争力和影响力。
rank: 4, distance: 1.1897048950195312, text: 东软集团在2023年通过了国家数据管理能力成熟度（DCMM）四级认证，数据管理能力达到量化管理级水平，并通过国产化信息系统集成和服务能力等级评估-优秀级（LS4）。公司发布了“下一代智能化解决方案”战略，以“AI”“数据要素”为核心，开启了以“主营业务高增长、经营高质量、行业高影响力”为目

# 对内容最相关的文档内部进行RAG

### 引入

In [5]:
import pdfplumber
import uts
from nltk.tokenize import sent_tokenize
import nltk

In [6]:
# 确保下载NLTK的punkt分词器数据
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 提取数据

In [7]:
import fitz  # PyMuPDF
import pymupdf4llm



In [8]:
text = ""
for data_name in data:
    doc = fitz.open(f"./data/{data_name}.pdf")
    for page in doc:
        output = page.get_text("blocks")
        previous_block_id = 0
        for block in output:
            if block[6] == 0:
                if block[4].replace('\n', '').replace(' ', '')!='':
                    text+=block[4].replace('\n', '')
                    text+='\n'

### 切分与chunking

In [9]:
from pysbd import Segmenter

segmenter = Segmenter()
setences = segmenter.segment(text)

In [10]:
setences_clear = []
for s in setences:
    s = s.replace('\n','').replace(' ', '')
    setences_clear.append(s)

In [11]:
model = uts.C99(window=2)
boundary = model.segment(setences_clear)
print(boundary)

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 

In [12]:
segment = []
current_segment = setences_clear[0]
for s, b in zip(setences_clear, boundary):
    if b==1:
        segment.append(current_segment)
        current_segment = s
    elif b==0:
        current_segment += s
# print(segment)

for data in segment:
    print(data)
    print('======')
    print()
print(len(segment))

请务必阅读正文之后的信息披露和法律声明

请务必阅读正文之后的信息披露和法律声明[Table_MainInfo]

公司研究/信息服务/软件证券研究报告南威软件（603636）公司年报点评2024年05月14日

[Table_InvestInfo]新签合同稳步增长，人工智能+数据服务战略快速推进

[Table_Summary]投资要点：新签合同稳步增长，外部环境承压背景下保持经营基本面。

公司发布2023年年报和2024年一季报。

2023年，公司实现营业收入16.77亿元，同比下降3.58%；归母净利润5629万元，同比下降59.18%；扣非归母净利润-3891万元，同比下降60.31%。2024年Q1，公司实现营业收入1.08亿元，同比下降26.97%，实现归母净利润-6748万元，2023年同期为-2178万元。

值得注意的是，2023年，公司新签合同签订额25.37亿元，同比增长4.92%；新增中标项目金额26.67亿元，同比增长7.07%；公司项目回款额13.72亿元，同比增长18.42%。从公司2023年的整体经营情况看，公司在宏观经济增速放缓和政府财政压力加大的大环境下，营业收入略微下降，保持了经营基本面，公司全国化布局与聚焦主营产品的经营策略产生了较好成效，实现了销售合同总额增长，销售结构进一步优化，福建省外销售额占比达76%，销售毛利润同比上涨，彰显了公司在全国市场的影响力与自主核心的软件竞争实力的提升。

但由于公司产品化转型与全国市场布局需要大量的前期投入，导致管理费用与业务费用增加，对2023年利润造成了一定影响，而销售合同额转化为营业收入需要一定周期，无法完全在当年体现，从而使得公司营业收入略微下降、利润下滑。人工智能+数据服务新战略奠定公司长期向好趋势。2023年公司对战略定位

进行调整，明确将人工智能、数据服务作为公司发展的新战略方向，确定了“南威软件是一家人工智能与数据服务的领先企业，致力于推动全球数字政府、公共安全、社会治理和智慧产业的创新发展”的新战略定位。基于此，公司于2023年成立了人工智能研发中心，大量投入行业人工智能的研发，成为2023年全国首个发布政府行业大模型（“白泽政务大模型”）的公司，同时，公司压强数据要素操作系统的研发，在国内率先发布数据要素操作系统。

这两方面的研发投入导致公司的费用增加，进而

### 寻找相似语句

In [13]:
import torch

from FlagEmbedding import FlagModel
model = FlagModel('BAAI/bge-large-zh-v1.5', 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True)
print("模型加载")
embeddings = model.encode(segment)

# print(embeddings)
# print(embeddings.shape)

模型加载


pre tokenize: 100%|██████████| 14/14 [00:00<00:00, 134.88it/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 14/14 [00:14<00:00,  1.03s/it]


In [14]:
# -*- coding: utf-8 -*-
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import json
import numpy as np
import faiss


nlist = 5  # 聚类中心的数量
quantizer = faiss.IndexFlatL2(1024)  # 基础索引
faiss_index = faiss.IndexIVFFlat(quantizer, 1024, nlist, faiss.METRIC_L2)


faiss_index.train(embeddings)
faiss_index.add(embeddings)

# 查询语句
query = question


embeddings_query = model.encode(query)
# print(embeddings_query.shape)

distances, doc_indices = faiss_index.search(embeddings_query, 10)


knowledge = []
print(f"query: {query}")
print("recall result: ")
print(doc_indices.shape, distances.shape)
for i in range(doc_indices.shape[0]):
    print('==== query:',query[i])
    for j, sent_index in enumerate(doc_indices[i]):
        print(f"rank: {j}, distance: {distances[i][j]}, text: {segment[sent_index]}")
        knowledge.append(segment[sent_index])
# reset index
# faiss_index.reset()


query: ['福昕软件在全球PDF行业中的竞争地位如何？']
recall result: 
(1, 10) (1, 10)
==== query: 福昕软件在全球PDF行业中的竞争地位如何？
rank: 0, distance: 0.37854963541030884, text: 福昕软件专注于PDF领域，高度产品化，具备全球竞争力。福昕软件自成立
rank: 1, distance: 0.5073010921478271, text: Adobe为绝对龙头，福昕在Kofax、Nitro、PDFelement（万兴）等二线厂商中处于领先位置。
rank: 2, distance: 0.5243108868598938, text: 福昕软件（688095.SH）：具备全球竞争力的版式软件领军企业敬请参阅报告结尾处的免责声明东方财智兴盛之源图18：Adobe与IDC对2024年文档云（PDF业务）潜在市场规模预测（美元）.............................................................
rank: 3, distance: 0.5392012000083923, text: PDF电子文档领域发展日趋成熟，全球维度逐步形成了以Adobe、福昕（Foxit）、Kofax（收购NuancePDF业务）、Nitro等为代表的知名品牌。Adobe为绝对龙头，福昕在Kofax、Nitro、PDFelement（万兴）等二线厂商中处于领先位置。公司不考虑潜在市场规模，从存量参与者的维度来看，Adobe作为全球龙头其PDF业务体量在2022年近28亿美元，规模在福昕软件的30倍以上，且当前仍保持较高的增速，即使不考虑新技术带来的提价预期、功能增长、非付费及盗版用户转化等，二线厂商具有较大发展空间。
rank: 4, distance: 0.5593022108078003, text: 当前Adobe在PDF端收入体量为福昕30余倍，不考虑增量客户与增量场景的情况下，撬动Adobe极低比例的客户即有望帮助福昕
rank: 5, distance: 0.5931335687637329, text: 经过多年国际市场的充分竞争，福昕软件已沉淀相当部分核心客户资源，建立良好品牌影响力，持续争取更多通用PDF软件的市场份额；

# 将retrive的结果当做prompt，进行agent

In [15]:
import ollama

client = ollama.Client(host='http://localhost:11434')


def get_completion(prompt, model="qwen2.5"):  # gpt-3.5-turbo
    messages = [{"role": "user", "content": prompt}]
    response =client.chat(
        model=model,
        messages=messages,
        #temperature=0.5, 
    )
    return response['message']['content']

In [16]:
setting = '''根据专家定义，我们将企业文化分为十类，分别是：
1.适应性：文化强调文化的转换、适应或对变化的抵制。例如，"文化变迁"、"文化转型"、"文化重置"等；
2.以客户为导向：以销售或客户关系为优先的文化。例如，"销售文化"，"以客户为中心"和"以品牌为中心"；
3.创新：强调创新、成长和创业的文化。例如，"创业型"、"创新型"、"成长型"、"技术驱动型"等；
4.诚信：重视道德行为、公正实践和社区关系的文化。例如，"开明好客"，"注重ESG "，"公平的商业行为"和"责备文化"；
5.经营导向：强调效率、生产率和成本控制的文化。例如，"分散化"、"效率中心"和"成本意识"；
6.以人为本：以员工福祉、多样性、包容、赋权和人才成长为核心的文化。例如，"以员工为中心"，"多样化和包容性"，以及"以人才为中心"；
7.结果导向：注重绩效、竞争力和结果的文化。例如，"竞争性"、"绩效驱动"、"绩效薪酬"等；
8.风险控制：强调风险管理、信用和金融审慎的文化。例如，"保守的信用文化"，"风险规避"和"合规文化"；
9.团队合作：强调协作、整合和团队导向的文化。例如，"协作"、"整合"、"文化契合"和"伙伴关系导向"；
10.其他：这个类别包含各种不特定的、出现频率较低的文化价值观，或者那些不容易被纳入上述类别的文化价值观。例如，"积极文化"和"独特文化"'''

agent_role = ["你是世界级的企业文化分析师，现在你需要从适应性文化的角度出发，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从客户为导向文化的角度出发，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从创新文化的角度出发，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从诚信文化的角度出发，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从经营导向文化的部分，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从以人为本文化的部分，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从结果导向文化的部分，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从风险控制文化的部分，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从团队合作文化的部分，根据下述已知知识分析给出的问题",
              "你是世界级的企业文化分析师，现在你需要从其他文化的部分，根据下述已知知识分析给出的问题"]

In [17]:
response = []
for i in range(10):
    prompt = f"{agent_role[i]}。对于企业文化的定义为：{setting}。当前问题为：{question}。目前已知知识为：{knowledge}。"
    response.append(get_completion(prompt)) 
    print(response[i])

根据提供的信息和问题背景，我们可以分析福昕软件在全球PDF行业的适应性文化表现。适应性文化强调文化对于变化和转型的支持与接纳。

1. **技术变革的应对**：
   - 福昕软件专注于PDF领域的技术创新，并且在人工智能（AI）领域有所布局，特别是在文档理解、总结、内容改写生成等方面。这表明公司能够快速适应技术的变化。
   - 公司也在尝试云化转型，并通过API调用降低成本来提升产品力，这种策略显示了福昕对于新兴技术和市场趋势的适应性。

2. **市场定位与竞争**：
   - 福昕软件在全球PDF市场上处于领先地位，在Kofax、Nitro等二线厂商中占据了优势地位。这意味着公司能够在全球范围内调整和优化其市场策略。
   - 虽然Adobe是绝对龙头，但福昕软件通过不断争取市场份额，建立品牌影响力，并且通过收购国内OFD业务的股权来拓展业务范围，显示出公司在面对强大竞争对手时的灵活性。

3. **产品多样化与市场扩展**：
   - 公司开发了包括通用软件产品、开发平台与工具、企业文档自动化解决方案以及PDF相关独立产品在内的较为完善的产品体系。这种多样化的战略有助于福昕在不同客户群体之间寻找新的增长点，进一步体现了其对市场需求变化的适应性。
   
4. **品牌影响力和客户关系**：
   - 福昕软件已经沉淀了一定的核心客户资源，并在全球范围内建立了良好的品牌影响力。这表明公司能够通过高质量的产品和服务来保持与客户的长期合作关系。

综上所述，福昕软件表现出较强的适应性文化特征。面对市场的技术革新、竞争压力以及客户需求的变化，该公司能够灵活调整自身战略和产品方向，以确保在PDF行业中的竞争力。这种文化和行为模式有助于企业更好地应对未来可能发生的各种变化。
根据您提供的已知知识，我们可以从“以客户为导向”的企业文化角度来分析福昕软件在全球PDF行业中的竞争地位。

### 福昕软件的竞争优势

1. **全球竞争力与品牌影响力**：
   - 文档云（PDF业务）市场潜力巨大。Adobe为龙头，但福昕在Kofax、Nitro等二线厂商中处于领先位置。
   - 全球范围内的市场份额增长依赖于核心客户资源和良好的品牌形象。

2. **技术创新能力**：
   - 投入研发并拥有大量专利（75项国内外发明专利）和技术著作权（294项），说明福昕在产品和